# Documentation

This notebook executes the `AngleAnalyzer` algorithm class. The `AngleAnalyzer` class is designed to analyze flight trajectories at a specific point by examining the angles between successive points. It reads flight data from a CSV file and calculates angles between points, determining if they form a straight line or a curve. Using provided thresholds, it extracts past and future coordinates, filters out zero angles, and performs both angle-based and linear regression analyses. These analyses help classify flight segments as either straight lines or curves. 

Input:
- `.csv` file of format (processed using `FileConverter`) `timestamp [UTC],relative altitude [m],horizontal velocity [m/s],vertical velocity [m/s],distance to takeoff [km],longitude,latitude`

Output:
- None

# Imports

In [ ]:
import os
import sys
import pandas as pd

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import helpers.data_visualizer as datavisualizer
import algorithms.angle_analyzer as angleanalyzer

# Variables

In [ ]:
# 200 expects linear (true), 270 expects curve (false), 400 expects overlap (false), 1400 expects straight line (true), 1320 expects end of curve (true), 1200 expects start of curve (false)
INDEX: int = 1400
CSV_FILE: str = "/Users/nicolas/Dropbox/3_Bildung/2 KZO/Stichwortverzeichnis/0 Maturarbeit/6 Wettbewerbe/1_SJf/3_Coaching/2_Projekt/software/flight-analyzer/docs/datasets/tracklogs/2_normalized/20211016_tracklog-normalized_nicolas-huber.csv"

# Initialisation

In [ ]:
Analyzer: angleanalyzer.AngleAnalyzer = angleanalyzer.AngleAnalyzer(
    csv_file=CSV_FILE,
    latest_threshold=constants.ANGLE_PAST_THRESHOLD,
    future_threshold=constants.ANGLE_FUTURE_THRESHOLD,
    angle_threshold=constants.ANGLE_THRESHOLD,
    linear_regression_threshold=constants.LINEAR_REGRESSION_THRESHOLD
)
Visualizer: datavisualizer.DataVisualizer = datavisualizer.DataVisualizer()

# Data Import

In [ ]:
data: pd.DataFrame = Analyzer.read_csv_file()

print("lenght: " + str(len(data)))
print()
print(data)

# Analyze Coordintes

In [ ]:
latest_coordinates: pd.DataFrame = Analyzer.extract_latest_coordinates(df=data, i=INDEX)
future_coordinates: pd.DataFrame = Analyzer.extract_future_coordinates(df=data, i=INDEX)

print("Past Coordinates")
print("--> lenght: " + str(len(latest_coordinates)))
print()
print(latest_coordinates)
print()

print("Future Coordinates")
print("--> lenght: " + str(len(future_coordinates)))
print()
print(future_coordinates)

# Calculate Angles

In [ ]:
angles_past: pd.DataFrame = Analyzer.cut_zero_angles(
    Analyzer.calculate_angles(df=latest_coordinates)
)
angles_future: pd.DataFrame = Analyzer.cut_zero_angles(
    Analyzer.calculate_angles(df=future_coordinates)
)

print("Past Angles")
print("--> lenght: " + str(len(angles_past)))
print()
print(angles_past)

print()

print("Future Angles")
print("--> lenght: " + str(len(angles_future)))
print()
print(angles_future)

# Visualisation

In [ ]:
Visualizer.visualize_points_colored(df=data, relative=INDEX)
Visualizer.visualize_points_2d(df=latest_coordinates, relative=0, linear=True, title="Vergangenheit")
Visualizer.visualize_points_2d(df=future_coordinates, relative=0, linear=True, title="Zukunft")
Visualizer.visualize_angles(past_angles=angles_past, future_angles=angles_future)

# Analysis

In [ ]:
status_angle_past: bool = Analyzer.analyze_angles(angles=angles_past)
status_angle_future: bool = Analyzer.analyze_angles(angles=angles_future)
(
    status_regression_past,
    slope_past,
    intercept_past,
    r_value_past,
    p_value_past,
    std_err_past,
) = Analyzer.analyze_linear_regression(df=latest_coordinates)
(
    status_regression_future,
    slope_future,
    intercept_future,
    r_value_future,
    p_value_future,
    std_err_future,
) = Analyzer.analyze_linear_regression(df=future_coordinates)

print("Angle Analysis")
print("--> Past: " + str(status_angle_past))
print("--> Future: " + str(status_angle_future))

print()

print("Past Linear Regression")
print("--> Status: " + str(status_regression_past))
print("--> Slope: " + str(slope_past))
print("--> Intercept: " + str(intercept_past))
print("--> R-Value: " + str(r_value_past))
print("--> P-Value: " + str(p_value_past))
print("--> Standard Error: " + str(std_err_past))

print()

print("Future Linear Regression")
print("--> Status: " + str(status_regression_future))
print("--> Slope: " + str(slope_future))
print("--> Intercept: " + str(intercept_future))
print("--> R-Value: " + str(r_value_future))
print("--> P-Value: " + str(p_value_future))
print("--> Standard Error: " + str(std_err_future))

print()

print("Data Analysis")
print(
    f"--> Status: {Analyzer.analyze_data(status_angle_past, status_regression_past, status_angle_future, status_regression_future)}"
)

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")